## Handoffs

A way for an agent to invoke another agent

In [ ]:
from IPython.display import Markdown
from pydantic import BaseModel
from agents import Agent, Runner, ModelSettings

class Tutorial(BaseModel):
  outline: str
  tutorial: str

model_settings = ModelSettings(
  temperature=0.1
)

tutorial_generator = Agent(
  name="Tutorial Generator",
  instructions=
    "Given a programming topic and an outline, your job is to generate code snippets for each section of the outline. " +
    "Format the tutorial in Markdown using a mix of text for explanation and code snippets for examples. " +
    "Where it makes sense, include comments in the code snippets to further explain the code.",
  model_settings=model_settings,
  output_type=Tutorial,
)

outline_builder = Agent(
  name="Outline Builder",
  instructions=
    "Given a particular programming topic, your job is to help come up with a tutorial. You will do that by crafting an outline. " +
    "After making the outline, hand it to the tutorial generator agent.",
  model_settings=model_settings,
  handoffs=[tutorial_generator],
)

tutorial_responses = await Runner.run(outline_builder, "Creating a Python Tutorial on Data Analysis with Pandas")
#display(Markdown(tutorial_responses.final_output))
print(tutorial_responses.final_output)

## Triage agent

This agent is used to triage questions. It will determine if the question is simple enough to answer directly, or if it should be handed off to a more specialized agent.

In [ ]:
from agents import Agent, Runner, handoff, RunContextWrapper

history_tutor_agent = Agent(
  name="History Tutor",
  handoff_description="Specialist agent for historical questions",
  instructions="You provide assistance with historical queries. Explain important events and context clearly.",
  model="o4-mini"
)

math_tutor_agent = Agent(
  name="Math Tutor",
  handoff_description="Specialist agent for mathematical problems, concepts, and questions",
  instructions="You provide assistance with mathematical queries. Solve problems step-by-step and explain concepts.",
  model="o4-mini"
)

def on_math_handoff(ctx: RunContextWrapper[None]):
  print("Handoff to math tutor agent")

def on_history_handoff(ctx: RunContextWrapper[None]):
  print("Handoff to history tutor agent")

triage_agent = Agent(
  name="Triage Agent",
  instructions="You determine which agent to use based on the user's homework questions. " +
  "If neither agent is relevant, provide a general response.",
  handoffs=[handoff(history_tutor_agent, on_handoff=on_history_handoff), handoff(math_tutor_agent, on_handoff=on_math_handoff)],
  model="o4-mini"
)


In [ ]:
result = await Runner.run(triage_agent, "How do I add 2 and 2?")
display(Markdown(result.final_output))

In [ ]:
result = await Runner.run(triage_agent, "What was the significance of the Battle of Hastings?")
result.final_output

In [ ]:
from pydantic import BaseModel
from agents import Agent, Runner, RunContextWrapper, function_tool, handoff

class ManagerEscalation(BaseModel):
  issue: str  # the issue being escalated
  why: str # why can you not handle it? Used for training in the future 

@function_tool
def create_ticket(issue: str):
  """
  Create a ticket in the system for an issue to be resolved.
  """
  print(f"Creating ticket for issue: {issue}")
  return f"Ticket created. ID: 12345"
  # In a real-world scenario, this would interact with a ticketing system

manager_agent = Agent(
  name="Manager",
  handoff_description="Handles escalated issues that require managerial attention.",
  instructions=(
    "You handle escalated customer issues that the initial customer service agent could not resolve. "
    "You will receive the issue and the reason for escalation. If the issue cannot be immediately resolved for the " 
    "customer, create a ticket in teh system and inform the customer."
  ),
  tools=[create_ticket],
)

def on_manager_handoff(ctx: RunContextWrapper[None], input: ManagerEscalation):
  print("Escalating to manager agent: ", input.issue)
  print("Reason for escalation: ", input.why)
  # here we might store the escalation in a database of log it for future reference

customer_service_agent = Agent(
  name="Customer Service",
  instructions="You assist customers with general inquiries and basic troubleshooting. " +
               "If the issue cannot be resolved, escalate it to the Manager along with the reason why you cannot fix the issue yourself.",
  handoffs=[handoff(
    agent=manager_agent,
    input_type=ManagerEscalation,
    on_handoff=on_manager_handoff
  )],
)

result = await Runner.run(customer_service_agent, "I want a refund, but your system won't let me process it. The website is just blank for me.")
result.final_output